# 数据清洗 #

In [2]:
#coding=utf-8
import time,re
import pandas as pd
import numpy as np
from pymongo import MongoClient
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties


shijian=time.strftime('%Y-%m-%d',time.localtime(time.time()))
font_set = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=8) #用于在散点图中输出中文
client=MongoClient('mongodb://root:' + '5768116' + '@121.196.220.14')
db = client.jishi         #得到数据库
collection = db.jishi      #得到数据集合
cursor = collection.find()#
df2 = pd.DataFrame(list(cursor))

df2=df2.fillna('NAN')  #把日期列的nan 数值用“NAN”填充，统一空值


df2=df2[(df2['页面'] !='NAN') & (df2['登录日期'] !='NAN')]

dfxint=df2['_id'].groupby([df2['登录日期'],df2['页面'],df2['ip']]).size()

dfxint2=df2['_id'].groupby([df2['登录日期'],df2['ip']]).size()

dfxint2=dfxint2.reset_index(drop = False)
dfxint2['登录日期']=dfxint2['登录日期'].astype('str')
dfxint2=dfxint2[dfxint2['登录日期']>='2020-10-10']


dfxint=dfxint.reset_index(drop = False)
dfxint['登录日期']=dfxint['登录日期'].astype('str')
dfxint=dfxint[dfxint['登录日期']>='2020-10-10']
print(dfxint)
dfxint.to_excel('77.xls')
print(len(dfxint2))
print(time.strftime('%Y-%m-%d %H:%M:%S'))

             登录日期     页面               ip  _id
18233  2020-10-10   人行数据      1.85.217.84    1
18234  2020-10-10     外汇   175.184.166.51    1
18235  2020-10-10   存单分析  115.195.192.214    1
18236  2020-10-10   存单分析    182.88.77.113    1
18237  2020-10-10   理财分析       36.32.3.53    1
18238  2020-10-10  票交所数据    171.34.176.42    1
18239  2020-10-10  票交所数据  220.181.108.160    1
18240  2020-10-10   票据分析   171.216.238.89    1
18241  2020-10-10   票据分析     42.236.10.75    1
18242  2020-10-10   票据分析     42.236.10.93    1
18243  2020-10-10   资金分析    203.208.60.13    1
18244  2020-10-10   资金分析    203.208.60.27    1
18245  2020-10-10   资金分析  220.200.156.148    1
18246  2020-10-10   金融资讯  114.101.153.112    1
18247  2020-10-11   交易日报   220.181.108.79    1
18248  2020-10-11   理财分析  220.181.108.111    1
18249  2020-10-11   票据分析      211.95.50.4    1
18250  2020-10-11   资金分析   203.208.60.114    1
18251  2020-10-11   资金分析    203.208.60.75    1
18252  2020-10-11   资金分析    203.208.60.96    1
18253  2020-1

In [3]:
from flask import Flask,render_template,redirect,url_for,jsonify,request,make_response,session

url0='http://www.51tradecloud.com/'
maxTryNum=10
import urllib.request
url0='http://www.51tradecloud.com/'
maxTryNum=10
url = url0+'g/1'
print(url)
for tries in range(maxTryNum):
      try:
                      res_data = urllib.request.urlopen(url,timeout=25)
                      res_data = res_data.read()
                      res_data=res_data.decode('utf-8')
                      result= pd.read_json(res_data,typ='frame')  
                      break
      except:
                      if tries < (maxTryNum - 1):
                          continue
                      else:
                          result=pd.DataFrame({})
                          break
if len(result)==0:
                 cundanz='网络差，请稍后重试'

result = result.sort_values(by=['月份'], ascending=True)
result=result.drop_duplicates(['月份'],keep='last')   #删除连接中的重复行
result['月份']=result['月份'].astype('str')  

result['去年同比流通中货币（M0）']=result['流通中货币（M0）'].shift(12)
result['去年同比货币（M1）']=result['货币（M1）'].shift(12)
result['去年同比货币和准货币（M2）']=result['货币和准货币（M2）'].shift(12)
result['M0同比增长率(%)']=round((result['流通中货币（M0）']-result['去年同比流通中货币（M0）'])/result['去年同比流通中货币（M0）']*100,2)
result['M1同比增长率(%)']=round((result['货币（M1）']-result['去年同比货币（M1）'])/result['去年同比货币（M1）']*100,2)
result['M2同比增长率(%)']=round((result['货币和准货币（M2）']-result['去年同比货币和准货币（M2）'])/result['去年同比货币和准货币（M2）']*100,2)
result=result.fillna(value='')  #先把NAN提成0
result=result[result['去年同比流通中货币（M0）']!='']
print(result)


http://www.51tradecloud.com/g/1


KeyError: '月份'

In [3]:

#自定义条件替换
df2.业务方向=df2.业务方向.apply(lambda x: '收票' if x=='shou'  else x)
df2.业务方向=df2.业务方向.apply(lambda x: '出票' if x=='chu'  else x)
df2.业务方向=df2.业务方向.apply(lambda x: '收证' if x=='shoufu'  else x)
df2.业务方向=df2.业务方向.apply(lambda x: '出证' if x=='chufu'  else x)
df2.业务方向=df2.业务方向.apply(lambda x: '收存单' if x=='shoucun'  else x)
df2.业务方向=df2.业务方向.apply(lambda x: '出存单' if x=='chucun'  else x)
df2.业务方向=df2.业务方向.apply(lambda x: '收理财' if x=='shouli'  else x)
df2.业务方向=df2.业务方向.apply(lambda x: '出理财' if x=='chuli'  else x)
print(df2.业务方向.value_counts())#查看每个值的计数。

#把频率小于2的值替换
#aa=dict(zip(*np.unique(df2.日期,return_counts=True)))
#df2.日期=df2.日期.apply(lambda x: '2017nian' if aa[x]<2 else x)




收票     15993
出票      7035
NAN     5354
收存单     2070
出存单       71
收证        65
出证        56
收理财       27
出理财       20
Name: 业务方向, dtype: int64


In [4]:
print(df2.业务方向.value_counts())#查看每个值的计数。
print(df2.业务方向.value_counts().argsort())#查看计数评率的排序

收票     15993
出票      7035
NAN     5354
收存单     2070
出存单       71
收证        65
出证        56
收理财       27
出理财       20
Name: 业务方向, dtype: int64
收票     8
出票     7
NAN    6
收存单    5
出存单    4
收证     3
出证     2
收理财    1
出理财    0
Name: 业务方向, dtype: int64


## 2.选择一：用groupby来细分业务方向 ##

In [4]:
#登录日期分析
import xlrd,xlwt
df2['登录日期'] = pd.to_datetime(df2['登录日期']).astype('str')  #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式

dfxint=df2['点击时间'].groupby([df2['登录日期'],df2['页面']]).size().unstack('页面')
dfxint=dfxint.reset_index(drop = False)
print(dfxint)
#dfxint.to_excel('dfxint.xls')


dfxint2=df2['点击时间'].groupby([df2['登录日期'],df2['页面'],df2['ip']]).size().unstack('页面')
dfxint2=dfxint2.reset_index(drop = False)
print(dfxint2)
#dfxint.to_excel('dfxint.xls')

页面                         登录日期    下载   业务撮合  人行数据  原创文章  存单分析  理财分析  票据分析  \
0    1970-01-01 00:25:51.571200   1.0    NaN   NaN   1.0   NaN   NaN   NaN   
1    1970-01-01 00:25:51.657600   NaN    NaN   NaN   NaN   NaN   NaN  14.0   
2    2019-03-03 00:00:00.000000   1.0    NaN   NaN   1.0   NaN   NaN   1.0   
3    2019-03-04 00:00:00.000000   6.0  548.0   NaN   3.0   3.0   2.0  35.0   
4    2019-03-05 00:00:00.000000   1.0  386.0   NaN   1.0   5.0   3.0  19.0   
5    2019-03-06 00:00:00.000000   NaN  330.0   NaN   NaN   8.0   5.0  23.0   
6    2019-03-07 00:00:00.000000   1.0  291.0   NaN   NaN   5.0   1.0  19.0   
7    2019-03-08 00:00:00.000000   3.0  286.0   NaN   1.0   7.0   1.0  23.0   
8    2019-03-09 00:00:00.000000   2.0    1.0   NaN   1.0   4.0   6.0   2.0   
9    2019-03-10 00:00:00.000000   NaN    2.0   NaN   NaN   5.0   4.0   NaN   
10   2019-03-11 00:00:00.000000   1.0  421.0   NaN   1.0   6.0   2.0  22.0   
11   2019-03-12 00:00:00.000000  11.0  279.0   NaN   NaN   3.0  

In [9]:
dfxin1=df2['点击时间'].groupby([df2['用户名'],df2['业务方向']]).size().unstack('业务方向')
print(dfxin1)

业务方向            chu     shou  shoucun  shouli   兴业   农商    城商    平安   广发   招行  \
用户名                                                                             
13095775559    76.0    172.0     39.0     NaN  NaN  NaN   NaN   NaN  NaN  NaN   
13616512098     3.0     51.0      NaN     1.0  NaN  NaN   NaN   NaN  NaN  NaN   
13758850838    64.0    111.0     11.0     NaN  NaN  NaN   NaN   NaN  NaN  NaN   
13813968884    10.0     15.0      3.0     NaN  3.0  2.0   2.0   4.0  1.0  2.0   
13913119224     NaN      1.0      NaN     NaN  NaN  NaN   NaN   NaN  NaN  NaN   
13953351737    21.0     97.0      NaN     NaN  2.0  8.0   9.0   4.0  3.0  2.0   
13966238885     NaN      4.0      NaN     NaN  NaN  NaN   NaN   NaN  NaN  NaN   
15392488405    44.0    227.0     70.0     NaN  3.0  4.0  13.0   6.0  2.0  5.0   
15800410659    17.0     95.0      7.0     NaN  NaN  NaN   NaN   NaN  NaN  NaN   
15823526396     1.0      NaN      1.0     NaN  NaN  NaN   NaN   NaN  NaN  NaN   
15852663602     NaN      5.0

In [4]:
dfxin=df2['点击时间'].groupby([df2['用户名'],df2['业务方向']]).size()
dfxin=dfxin.reset_index(drop = False)
dfxin2=dfxin['业务方向'].groupby([dfxin['用户名']]).size()
dfxin3=dfxin['点击时间'].groupby([dfxin['用户名']]).mean()
dfxin4=dfxin['点击时间'].groupby([dfxin['用户名']]).std()
print(dfxin)
print(dfxin2)
print(dfxin3)
print(dfxin4)

             用户名 业务方向  点击时间
0                 NAN     4
1    13095775559  NAN    24
2    13095775559   出票     4
3    13095775559  收存单     1
4    13095775559   收票     8
5    13682273090  NAN     6
6    13682273090   收票     1
7    13917623677  NAN     2
8    15145671010  NAN     2
9    15145671010  收存单     1
10   15145671010   收票     8
11   15857176575  NAN    27
12   15857176575   出票    12
13   15857176575  收存单     8
14   15857176575   收票   102
15   16605596664  NAN     2
16   18161236815  NAN     8
17   18252487665  NAN    12
18   18252487665   出票     2
19   18252487665   出证     1
20   18252487665  收存单     5
21   18252487665   收票     2
22   18252487665   收证     1
23   18324499990  NAN    42
24   18324499990  出存单     1
25   18324499990   出票     2
26   18324499990  收存单     1
27   18324499990   收票     6
28  183244999990  NAN     1
29   18757098596  NAN     4
..           ...  ...   ...
36   18758599617   收票    12
37   18768189566  NAN     1
38   18768189566   收票     4
39   18806173545  NA

In [10]:
dfxin.index=dfxin.用户名
zong=pd.concat([dfxin1,dfxin2,dfxin3,dfxin4,dfxint1,dfxint2,dfxint3], axis=1)
columns=dfxin1.columns.values.tolist()+['使用的业务类型','平均点击次数','点击次数的标准差',' 最近使用日期','最早使用日期','使用天数']
print(columns[len(columns)-1])
zong.columns = columns
#zong.rename(columns = {len(columns)-3:columns[len(columns)-3], len(columns)-2: columns[len(columns)-2], len(columns)-1: columns[len(columns)-1]}, inplace=True)
# result.rename(columns={'time':'日期','time2':'时间','nickname':'微信名','hanglei1':'行类','content':'内容'}, inplace=True)
#zong(columns=columns)
print(zong)

使用天数
                chu     shou  shoucun  shouli   兴业   农商    城商    平安   广发   招行  \
用户名                                                                             
13095775559    76.0    172.0     39.0     NaN  NaN  NaN   NaN   NaN  NaN  NaN   
13616512098     3.0     51.0      NaN     1.0  NaN  NaN   NaN   NaN  NaN  NaN   
13758850838    64.0    111.0     11.0     NaN  NaN  NaN   NaN   NaN  NaN  NaN   
13813968884    10.0     15.0      3.0     NaN  3.0  2.0   2.0   4.0  1.0  2.0   
13913119224     NaN      1.0      NaN     NaN  NaN  NaN   NaN   NaN  NaN  NaN   
13953351737    21.0     97.0      NaN     NaN  2.0  8.0   9.0   4.0  3.0  2.0   
13966238885     NaN      4.0      NaN     NaN  NaN  NaN   NaN   NaN  NaN  NaN   
15392488405    44.0    227.0     70.0     NaN  3.0  4.0  13.0   6.0  2.0  5.0   
15800410659    17.0     95.0      7.0     NaN  NaN  NaN   NaN   NaN  NaN  NaN   
15823526396     1.0      NaN      1.0     NaN  NaN  NaN   NaN   NaN  NaN  NaN   
15852663602     NaN    

## 2.选择二： 用argsort和遍历的方式来细分业务方向 ##

In [6]:
renshu=df2.groupby(df2['用户名'])

In [7]:
import numpy as np
f_act=df2.业务方向.value_counts().argsort()
f_act=np.argsort(df2.业务方向.value_counts())

names=f_act.index.tolist()
names=['总数']+names+['类型数','平均数','标准差']
print(names)
print(f_act)

samples=[]
cont=0
for g in renshu:   #在用户ID中遍历  所有g都可以分为g[0]和g[1]
 #   if cont%10==0:
  #      print("%s from %s"%(cont,len(renshu)))
    gr=g[1]      #gr设定为除了id之外的所有行为

    l=[]     
    l.append(g[0]) #把id作为第一个项
    l.append(len(gr))  #把所有行为的总数作为第二个项
    c_act=[0]*len(f_act)  #把后面的所有行为的初始值设置为0
    for i,v in enumerate(gr.业务方向.values):  #遍历gr的业务方向列，如有有f_act中的行为就加1

        c_act[f_act[v]] +=1
        print(f_act[v])
        print(c_act)
        print(names[f_act[v]])
    _,c_act_uqc = np.unique(gr.业务方向.values,return_counts=True)
    print(_,c_act_uqc)
    print(c_act_uqc)
    print(c_act)
    c_act +=[len(c_act_uqc),np.mean(c_act_uqc),np.std(c_act_uqc)]
    l=l+c_act
    print(l)
    samples.append(l)
    cont +=1

['总数', '收票', '出票', 'NAN', '收存单', '出存单', '出证', '收证', '收理财', '出理财', '类型数', '平均数', '标准差']
收票     8
出票     7
NAN    6
收存单    5
出存单    4
出证     3
收证     2
收理财    1
出理财    0
Name: 业务方向, dtype: int64
6
[0, 0, 0, 0, 0, 0, 1, 0, 0]
出证
6
[0, 0, 0, 0, 0, 0, 2, 0, 0]
出证
6
[0, 0, 0, 0, 0, 0, 3, 0, 0]
出证
6
[0, 0, 0, 0, 0, 0, 4, 0, 0]
出证
['NAN'] [4]
[4]
[0, 0, 0, 0, 0, 0, 4, 0, 0]
['', 4, 0, 0, 0, 0, 0, 0, 4, 0, 0, 1, 4.0, 0.0]
6
[0, 0, 0, 0, 0, 0, 1, 0, 0]
出证
6
[0, 0, 0, 0, 0, 0, 2, 0, 0]
出证
6
[0, 0, 0, 0, 0, 0, 3, 0, 0]
出证
6
[0, 0, 0, 0, 0, 0, 4, 0, 0]
出证
6
[0, 0, 0, 0, 0, 0, 5, 0, 0]
出证
8
[0, 0, 0, 0, 0, 0, 5, 0, 1]
收理财
7
[0, 0, 0, 0, 0, 0, 5, 1, 1]
收证
7
[0, 0, 0, 0, 0, 0, 5, 2, 1]
收证
7
[0, 0, 0, 0, 0, 0, 5, 3, 1]
收证
6
[0, 0, 0, 0, 0, 0, 6, 3, 1]
出证
6
[0, 0, 0, 0, 0, 0, 7, 3, 1]
出证
6
[0, 0, 0, 0, 0, 0, 8, 3, 1]
出证
6
[0, 0, 0, 0, 0, 0, 9, 3, 1]
出证
6
[0, 0, 0, 0, 0, 0, 10, 3, 1]
出证
6
[0, 0, 0, 0, 0, 0, 11, 3, 1]
出证
6
[0, 0, 0, 0, 0, 0, 12, 3, 1]
出证
6
[0, 0, 0, 0, 0, 0, 13, 3, 1]
出证
6
[0, 0, 0, 0, 0,

7
[0, 0, 0, 0, 0, 0, 34, 21, 5]
收证
6
[0, 0, 0, 0, 0, 0, 35, 21, 5]
出证
6
[0, 0, 0, 0, 0, 0, 36, 21, 5]
出证
6
[0, 0, 0, 0, 0, 0, 37, 21, 5]
出证
6
[0, 0, 0, 0, 0, 0, 38, 21, 5]
出证
6
[0, 0, 0, 0, 0, 0, 39, 21, 5]
出证
6
[0, 0, 0, 0, 0, 0, 40, 21, 5]
出证
6
[0, 0, 0, 0, 0, 0, 41, 21, 5]
出证
6
[0, 0, 0, 0, 0, 0, 42, 21, 5]
出证
6
[0, 0, 0, 0, 0, 0, 43, 21, 5]
出证
7
[0, 0, 0, 0, 0, 0, 43, 22, 5]
收证
6
[0, 0, 0, 0, 0, 0, 44, 22, 5]
出证
6
[0, 0, 0, 0, 0, 0, 45, 22, 5]
出证
7
[0, 0, 0, 0, 0, 0, 45, 23, 5]
收证
6
[0, 0, 0, 0, 0, 0, 46, 23, 5]
出证
7
[0, 0, 0, 0, 0, 0, 46, 24, 5]
收证
6
[0, 0, 0, 0, 0, 0, 47, 24, 5]
出证
6
[0, 0, 0, 0, 0, 0, 48, 24, 5]
出证
7
[0, 0, 0, 0, 0, 0, 48, 25, 5]
收证
6
[0, 0, 0, 0, 0, 0, 49, 25, 5]
出证
6
[0, 0, 0, 0, 0, 0, 50, 25, 5]
出证
7
[0, 0, 0, 0, 0, 0, 50, 26, 5]
收证
7
[0, 0, 0, 0, 0, 0, 50, 27, 5]
收证
6
[0, 0, 0, 0, 0, 0, 51, 27, 5]
出证
7
[0, 0, 0, 0, 0, 0, 51, 28, 5]
收证
6
[0, 0, 0, 0, 0, 0, 52, 28, 5]
出证
7
[0, 0, 0, 0, 0, 0, 52, 29, 5]
收证
7
[0, 0, 0, 0, 0, 0, 52, 30, 5]
收证
7
[0, 0, 0, 0, 0, 0,

8
[0, 0, 0, 0, 0, 2, 265, 96, 30]
收理财
7
[0, 0, 0, 0, 0, 2, 265, 97, 30]
收证
7
[0, 0, 0, 0, 0, 2, 265, 98, 30]
收证
6
[0, 0, 0, 0, 0, 2, 266, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 267, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 268, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 269, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 270, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 271, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 272, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 273, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 274, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 275, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 276, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 277, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 278, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 279, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 280, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 281, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 282, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 283, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 284, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 285, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 286, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 287, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 288, 98, 30]
出证
6
[0, 0, 0, 0, 0, 2, 289, 98, 30]
出证


[0, 0, 0, 0, 0, 7, 4, 1, 48]
出存单
8
[0, 0, 0, 0, 0, 7, 4, 1, 49]
收理财
5
[0, 0, 0, 0, 0, 8, 4, 1, 49]
出存单
8
[0, 0, 0, 0, 0, 8, 4, 1, 50]
收理财
8
[0, 0, 0, 0, 0, 8, 4, 1, 51]
收理财
8
[0, 0, 0, 0, 0, 8, 4, 1, 52]
收理财
8
[0, 0, 0, 0, 0, 8, 4, 1, 53]
收理财
8
[0, 0, 0, 0, 0, 8, 4, 1, 54]
收理财
8
[0, 0, 0, 0, 0, 8, 4, 1, 55]
收理财
8
[0, 0, 0, 0, 0, 8, 4, 1, 56]
收理财
8
[0, 0, 0, 0, 0, 8, 4, 1, 57]
收理财
5
[0, 0, 0, 0, 0, 9, 4, 1, 57]
出存单
8
[0, 0, 0, 0, 0, 9, 4, 1, 58]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 1, 59]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 1, 60]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 1, 61]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 1, 62]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 1, 63]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 1, 64]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 1, 65]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 1, 66]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 1, 67]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 1, 68]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 1, 69]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 1, 70]
收理财
7
[0, 0, 0, 0, 0, 9, 4, 2, 70]
收证
8
[0, 0, 0, 0, 0, 9, 4, 2, 71]
收理财
8
[0, 0, 0, 0, 0, 9, 4, 2, 72]
收理财
8
[0, 0, 0, 0, 0, 9, 4,

[0, 0, 0, 0, 0, 7, 41, 61, 122]
收理财
7
[0, 0, 0, 0, 0, 7, 41, 62, 122]
收证
8
[0, 0, 0, 0, 0, 7, 41, 62, 123]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 62, 124]
收理财
7
[0, 0, 0, 0, 0, 7, 41, 63, 124]
收证
8
[0, 0, 0, 0, 0, 7, 41, 63, 125]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 63, 126]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 63, 127]
收理财
7
[0, 0, 0, 0, 0, 7, 41, 64, 127]
收证
8
[0, 0, 0, 0, 0, 7, 41, 64, 128]
收理财
7
[0, 0, 0, 0, 0, 7, 41, 65, 128]
收证
8
[0, 0, 0, 0, 0, 7, 41, 65, 129]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 65, 130]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 65, 131]
收理财
7
[0, 0, 0, 0, 0, 7, 41, 66, 131]
收证
8
[0, 0, 0, 0, 0, 7, 41, 66, 132]
收理财
7
[0, 0, 0, 0, 0, 7, 41, 67, 132]
收证
8
[0, 0, 0, 0, 0, 7, 41, 67, 133]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 67, 134]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 67, 135]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 67, 136]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 67, 137]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 67, 138]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 67, 139]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 67, 140]
收理财
8
[0, 0, 0, 0, 0, 7, 41, 67, 141]
收理财
5
[0, 0, 0, 0, 0, 8,

...

8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6218]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6219]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6220]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6221]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6222]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6223]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6224]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6225]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6226]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6227]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6228]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6229]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6230]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6231]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6232]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6233]
收理财
8
[8, 15, 22, 29, 46, 813, 1280, 2460, 6234]
收理财
7
[8, 15, 22, 29, 46, 813, 1280, 2461, 6234]
收证
8
[8, 15, 22, 29, 46, 813, 1280, 2461, 6235]
收理财
6
[8, 15, 22, 29, 46, 813, 1281, 2461, 6235]
出证
8
[8, 15, 22, 29, 46, 

出证
8
[8, 16, 26, 32, 47, 834, 1441, 2534, 6453]
收理财
7
[8, 16, 26, 32, 47, 834, 1441, 2535, 6453]
收证
8
[8, 16, 26, 32, 47, 834, 1441, 2535, 6454]
收理财
5
[8, 16, 26, 32, 47, 835, 1441, 2535, 6454]
出存单
5
[8, 16, 26, 32, 47, 836, 1441, 2535, 6454]
出存单
7
[8, 16, 26, 32, 47, 836, 1441, 2536, 6454]
收证
8
[8, 16, 26, 32, 47, 836, 1441, 2536, 6455]
收理财
7
[8, 16, 26, 32, 47, 836, 1441, 2537, 6455]
收证
8
[8, 16, 26, 32, 47, 836, 1441, 2537, 6456]
收理财
7
[8, 16, 26, 32, 47, 836, 1441, 2538, 6456]
收证
6
[8, 16, 26, 32, 47, 836, 1442, 2538, 6456]
出证
5
[8, 16, 26, 32, 47, 837, 1442, 2538, 6456]
出存单
8
[8, 16, 26, 32, 47, 837, 1442, 2538, 6457]
收理财
8
[8, 16, 26, 32, 47, 837, 1442, 2538, 6458]
收理财
8
[8, 16, 26, 32, 47, 837, 1442, 2538, 6459]
收理财
5
[8, 16, 26, 32, 47, 838, 1442, 2538, 6459]
出存单
8
[8, 16, 26, 32, 47, 838, 1442, 2538, 6460]
收理财
6
[8, 16, 26, 32, 47, 838, 1443, 2538, 6460]
出证
5
[8, 16, 26, 32, 47, 839, 1443, 2538, 6460]
出存单
8
[8, 16, 26, 32, 47, 839, 1443, 2538, 6461]
收理财
7
[8, 16, 26, 32, 47, 8

收理财
5
[9, 16, 28, 34, 47, 850, 1508, 2585, 6647]
出存单
7
[9, 16, 28, 34, 47, 850, 1508, 2586, 6647]
收证
8
[9, 16, 28, 34, 47, 850, 1508, 2586, 6648]
收理财
8
[9, 16, 28, 34, 47, 850, 1508, 2586, 6649]
收理财
8
[9, 16, 28, 34, 47, 850, 1508, 2586, 6650]
收理财
8
[9, 16, 28, 34, 47, 850, 1508, 2586, 6651]
收理财
8
[9, 16, 28, 34, 47, 850, 1508, 2586, 6652]
收理财
5
[9, 16, 28, 34, 47, 851, 1508, 2586, 6652]
出存单
8
[9, 16, 28, 34, 47, 851, 1508, 2586, 6653]
收理财
8
[9, 16, 28, 34, 47, 851, 1508, 2586, 6654]
收理财
7
[9, 16, 28, 34, 47, 851, 1508, 2587, 6654]
收证
7
[9, 16, 28, 34, 47, 851, 1508, 2588, 6654]
收证
7
[9, 16, 28, 34, 47, 851, 1508, 2589, 6654]
收证
8
[9, 16, 28, 34, 47, 851, 1508, 2589, 6655]
收理财
8
[9, 16, 28, 34, 47, 851, 1508, 2589, 6656]
收理财
8
[9, 16, 28, 34, 47, 851, 1508, 2589, 6657]
收理财
8
[9, 16, 28, 34, 47, 851, 1508, 2589, 6658]
收理财
8
[9, 16, 28, 34, 47, 851, 1508, 2589, 6659]
收理财
8
[9, 16, 28, 34, 47, 851, 1508, 2589, 6660]
收理财
8
[9, 16, 28, 34, 47, 851, 1508, 2589, 6661]
收理财
7
[9, 16, 28, 34, 47

7
[9, 16, 29, 34, 49, 878, 1533, 2662, 6847]
收证
8
[9, 16, 29, 34, 49, 878, 1533, 2662, 6848]
收理财
8
[9, 16, 29, 34, 49, 878, 1533, 2662, 6849]
收理财
7
[9, 16, 29, 34, 49, 878, 1533, 2663, 6849]
收证
8
[9, 16, 29, 34, 49, 878, 1533, 2663, 6850]
收理财
8
[9, 16, 29, 34, 49, 878, 1533, 2663, 6851]
收理财
8
[9, 16, 29, 34, 49, 878, 1533, 2663, 6852]
收理财
8
[9, 16, 29, 34, 49, 878, 1533, 2663, 6853]
收理财
8
[9, 16, 29, 34, 49, 878, 1533, 2663, 6854]
收理财
8
[9, 16, 29, 34, 49, 878, 1533, 2663, 6855]
收理财
7
[9, 16, 29, 34, 49, 878, 1533, 2664, 6855]
收证
6
[9, 16, 29, 34, 49, 878, 1534, 2664, 6855]
出证
8
[9, 16, 29, 34, 49, 878, 1534, 2664, 6856]
收理财
8
[9, 16, 29, 34, 49, 878, 1534, 2664, 6857]
收理财
7
[9, 16, 29, 34, 49, 878, 1534, 2665, 6857]
收证
8
[9, 16, 29, 34, 49, 878, 1534, 2665, 6858]
收理财
7
[9, 16, 29, 34, 49, 878, 1534, 2666, 6858]
收证
7
[9, 16, 29, 34, 49, 878, 1534, 2667, 6858]
收证
7
[9, 16, 29, 34, 49, 878, 1534, 2668, 6858]
收证
8
[9, 16, 29, 34, 49, 878, 1534, 2668, 6859]
收理财
7
[9, 16, 29, 34, 49, 878, 1

7
[9, 16, 31, 35, 49, 898, 1605, 2737, 7010]
收证
6
[9, 16, 31, 35, 49, 898, 1606, 2737, 7010]
出证
8
[9, 16, 31, 35, 49, 898, 1606, 2737, 7011]
收理财
6
[9, 16, 31, 35, 49, 898, 1607, 2737, 7011]
出证
8
[9, 16, 31, 35, 49, 898, 1607, 2737, 7012]
收理财
8
[9, 16, 31, 35, 49, 898, 1607, 2737, 7013]
收理财
7
[9, 16, 31, 35, 49, 898, 1607, 2738, 7013]
收证
5
[9, 16, 31, 35, 49, 899, 1607, 2738, 7013]
出存单
8
[9, 16, 31, 35, 49, 899, 1607, 2738, 7014]
收理财
6
[9, 16, 31, 35, 49, 899, 1608, 2738, 7014]
出证
6
[9, 16, 31, 35, 49, 899, 1609, 2738, 7014]
出证
6
[9, 16, 31, 35, 49, 899, 1610, 2738, 7014]
出证
6
[9, 16, 31, 35, 49, 899, 1611, 2738, 7014]
出证
8
[9, 16, 31, 35, 49, 899, 1611, 2738, 7015]
收理财
6
[9, 16, 31, 35, 49, 899, 1612, 2738, 7015]
出证
8
[9, 16, 31, 35, 49, 899, 1612, 2738, 7016]
收理财
6
[9, 16, 31, 35, 49, 899, 1613, 2738, 7016]
出证
6
[9, 16, 31, 35, 49, 899, 1614, 2738, 7016]
出证
8
[9, 16, 31, 35, 49, 899, 1614, 2738, 7017]
收理财
6
[9, 16, 31, 35, 49, 899, 1615, 2738, 7017]
出证
7
[9, 16, 31, 35, 49, 899, 1615,

In [8]:
samples=np.array(samples)
print(samples)
samp_ar =samples[:, 1:].astype(np.float16)
samp_id =samples[:, 0]

col_names=[]
for i in range(len(samples[0])-1):
    col_names.append('c_'+str(i))
df_act=pd.DataFrame(samp_ar,columns=names)
df_act['id']=samp_id
df_act.index=df_act.id
print(df_act.head())

[['' '4' '0' '0' '0' '0' '0' '0' '4' '0' '0' '1' '4.0' '0.0']
 ['13095775559' '37' '0' '0' '0' '0' '0' '1' '24' '4' '8' '4' '9.25'
  '8.870597499605086']
 ['13682273090' '7' '0' '0' '0' '0' '0' '0' '6' '0' '1' '2' '3.5' '2.5']
 ['13917623677' '2' '0' '0' '0' '0' '0' '0' '2' '0' '0' '1' '2.0' '0.0']
 ['15145671010' '11' '0' '0' '0' '0' '0' '1' '2' '0' '8' '3'
  '3.6666666666666665' '3.091206165165235']
 ['15857176575' '149' '0' '0' '0' '0' '0' '8' '27' '12' '102' '4' '37.25'
  '38.04848880047669']
 ['16605596664' '2' '0' '0' '0' '0' '0' '0' '2' '0' '0' '1' '2.0' '0.0']
 ['18161236815' '8' '0' '0' '0' '0' '0' '0' '8' '0' '0' '1' '8.0' '0.0']
 ['18252487665' '23' '0' '0' '1' '1' '0' '5' '12' '2' '2' '6'
  '3.8333333333333335' '3.8908725099762504']
 ['18324499990' '52' '0' '0' '0' '0' '1' '1' '42' '2' '6' '5' '10.4'
  '15.90723106011854']
 ['183244999990' '1' '0' '0' '0' '0' '0' '0' '1' '0' '0' '1' '1.0' '0.0']
 ['18757098596' '56' '0' '0' '0' '0' '0' '4' '4' '45' '3' '4' '14.0'
  '17.9025

In [9]:
import numpy as np
ceshi=df2.点击时间.astype('str').values
print(ceshi)
ceshi=np.where(np.logical_and(ceshi=='10:26'),ceshi,ceshi)

['NAN' 'NAN' 'NAN' ... '09:13' '09:16' '09:16']


ValueError: invalid number of arguments

In [10]:
mpl.rcParams['font.sans-serif'] = ['SimHei']  #配置显示中文，否则乱码
mpl.rcParams['axes.unicode_minus']=False #用来正常显示负号，如果是plt画图，则将mlp换成plt
sns.pairplot(df, x_vars=['3个月存单利率','收票'], y_vars='足年国股',kind="reg", size=5, aspect=0.7)
plt.show()  #注意必须加上这一句，否则无法显示。


NameError: name 'mpl' is not defined

In [ ]:


#nsample = 100
x=df[['3个月存单利率','收票/出票']]  #如果样本是二维的不用用reshape，但如果是一维的是要用reshape
#x = np.linspace(0, 10, nsample)

beta=np.array([1,10])
e = np.random.normal(size=len(df))
#y = np.dot(X, beta) + e

y=df['足年国股利率变动']

from sklearn.linear_model import LinearRegression

lr=LinearRegression()
lr.fit(X,y)
#yh=lr.predict(x)  #.reshape(-1,1)
print(lr.intercept_)  #输出常数项
print(lr.coef_)  #输出截距  这个每次抽样都不一样，不一样的变动是多少？用confidence interval




# 建立训练数据和测试数据分开的回归 #

In [24]:
from sklearn.model_selection import train_test_split   #这里是引用了交叉验证 
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state=1)
lr=LinearRegression()
lr.fit(X_train,y_train)
#yh=lr.predict(x)  #.reshape(-1,1)
print(lr.intercept_)  #输出常数项
print(lr.coef_)  #输出截距  这个每次抽样都不一样，不一样的变动是多少？用confidence interval


0.010006079407746483
[ 0.          0.0354818  -0.00788247]


# sklearn的线性回归特别弱，所用要用statsmodels,来看相关的确定性 #

In [25]:
import statsmodels.api as sm
X=sm.add_constant(X)   #用来增加常数项
model=sm.OLS(y,X.astype(float)).fit()


In [26]:
print(model.params)
print(model.summary())  #R-squared，P

const        0.009019
3个月存单利率变动    0.047136
收票/出票       -0.007841
dtype: float64
                            OLS Regression Results                            
Dep. Variable:               足年国股利率变动   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     15.33
Date:                Mon, 25 Feb 2019   Prob (F-statistic):           4.83e-07
Time:                        22:46:11   Log-Likelihood:                 718.55
No. Observations:                 281   AIC:                            -1431.
Df Residuals:                     278   BIC:                            -1420.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------